# The Bad-Tempered Triangle
In which we speculate what a basic [chaos game](https://en.wikipedia.org/wiki/Chaos_game) sequence ought to sound like.

In [ ]:
from itertools import islice
import threading
import time

from ipycanvas import Canvas, hold_canvas
from IPython.display import display
import ipytone
import ipywidgets as widgets
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
from scipy import interpolate

First, find the vertices of a unit equilateral triangle.

The edge vectors will be the successive differences of vertices from [np.roll](https://numpy.org/doc/stable/reference/generated/numpy.roll.html).

The projections of a given point to the edges will the scalar products of the vectors from the vertices to the point and the edge vectors

In [ ]:
vertices = np.zeros((3, 2))

vertices[1, 0] = 0.5
vertices[1, 1] = np.sqrt(3.0) / 2.0
vertices[2, 0] = 1.0

edges = np.roll(vertices, 2, axis=0) - vertices

def projections(point):
    return np.sum((point - vertices) * edges, axis=1)

Pick a random vertex, take its mid-point to another random vertex, repeat to fade...

In [ ]:
def chaos_game():
    point = vertices[np.random.choice(np.arange(3))]
    while True:
        point = (point + vertices[np.random.choice(np.arange(3))]) / 2
        yield point

Return a series of images, tracing a given number of points at a time:
(The RGB values are mapped to the three projections of the points.)

In [ ]:
def chaos_images(size=256, step=50):
    height = int(size * np.sqrt(3) / 2)
    im = np.zeros((height, size, 3), dtype=np.uint8)
    size_ = size - 1
    height_ = height - 1
    points = chaos_game()
    while True:
        for point in islice(points, 0, step):
            x, y = (point * size_).astype(np.int32)
            im[height_-y, x] = (255 * projections(point)).astype(np.uint8)
        yield  Image.fromarray(im, 'RGB')

Quick-and-dirty UI, chaos-game for the use of...

In [ ]:
class ChaosPlayer(object):
    def __init__(self, size=512):
        self.ani_playing = False
        
        play_pause_button = widgets.Button(value=False, description='\u23EF')
        play_pause_button.style.font_size = '20pt'
        play_pause_button.on_click(self.ani_play_pause)
        
        self.speed_slider = widgets.IntSlider(value=10, min=5, max=100,
            step=5, description='speed', orientation='horizontal',
            disabled=False)

        ani_ctrl_box = widgets.HBox([play_pause_button, self.speed_slider])
        self.img_box = Canvas()
        
        self.width = size
        height = int(size * np.sqrt(3) / 2)
        self.canvas = Canvas(width=self.width, height=height)
        
        self.canvas.layout.width = '{}px'.format(self.width)
        self.canvas.layout.height = "{}px".format(height)
        
        self.img_box.fill_style = "black"
        self.img_box.fill_rect(0, 0, self.width, height)
        
        self.ani_box = widgets.VBox([ani_ctrl_box, self.img_box])

    def ani_worker(self):
        img_seq = chaos_images(size=512, step=self.speed_slider.value)
        while self.ani_playing:
            self.img_box.put_image_data(img_seq.__next__().__array__(), 0, 0)
            time.sleep(0.001)

    def ani_play_pause(self, _):
        self.ani_playing = not self.ani_playing
        self.speed_slider.disabled = self.ani_playing
        if self.ani_playing:
            thread = threading.Thread(target=self.ani_worker)
            thread.start()

    def run(self):
        display(self.ani_box)

ChaosPlayer().run()

From the chaos game sequence, yield the projections, their deltas, the cartesian point, the euclidean distance moved, and whether the projections crossed the centre of each edge.

In [ ]:
def chaos_seq():
    game = chaos_game()
    point = game.__next__()
    proj = projections(point)
    yield (proj, np.zeros((3,)), point, 0, (False, False, False))
    for next_point in game:
        next_proj = projections(next_point)
        triggers = (proj - 0.5) * (next_proj - 0.5) < 0
        yield (next_proj, next_proj - proj, next_point,
            np.sqrt(np.square(next_point - point).sum()),
            triggers
        )
        point = next_point
        proj = next_proj

In a vague concession to musicality, turn the projections into frequencies by interpolating between semitones for a given set of octaves. The *n*th key of a [piano keyboard](https://en.wikipedia.org/wiki/Piano_key_frequencies) is supposed to be of frequency:

$$\LARGE{2^{\frac{n-49}{12}}\times 440}$$

In [ ]:
def get_octave_map(octave=3, n_octaves=1):
    n_notes = n_octaves * 12
    n_notes_ = n_notes - 1
    notes = np.arange(n_notes)
    offset = 4 + (octave - 1) * 12
    freqs = np.power(2, (np.arange(offset, offset+n_notes) - 49) / 12) * 440
    interp = interpolate.interp1d(notes, freqs)
    
    def octave_map(pitches):
        return interp(pitches * n_notes_)
    
    return octave_map

Finally get around to playing with [Ipytone](https://ipytone.readthedocs.io/en/latest/index.html). Seems somewhat *wasteful* to have one envelope and filter per oscillator. Doomed to using [`time.sleep`](https://docs.python.org/3/library/time.html#time.sleep) and [`threading`](https://docs.python.org/3/library/threading.html) to sequence events, as the `step` method involves things other than [Ipytone](https://ipytone.readthedocs.io/en/latest/index.html) API calls. The signal path is:

* The projections of each point from the chaos game set the pitches of three sets of [detuned oscillators](https://ipytone.readthedocs.io/en/latest/_api_generated/ipytone.FatOscillator.html).
* The differences in the projections set the detune of each oscillator.
* The euclidean distance moved sets the filter cut-off.
* If the projections move past the half-way point of each edge, a [drum](https://ipytone.readthedocs.io/en/latest/_api_generated/ipytone.MembraneSynth.html) is triggered, at half the pitch of the corresponsing oscillator. 

In [ ]:
class ChaosGameSynth(object):
    
    def __init__(self, octave=3, n_octaves=1, bpm=240, gate=0.8, drum_gate=0.8, cut_off=200, depth=100, detune=200):
        self.interval = 60.0 / bpm
        self.gate = gate
        self.drum_gate = drum_gate
        self.cut_off = cut_off
        self.depth = depth
        self.detune = detune
        
        self.mute_drums = False
        self.playing = False
        
        self.oscs = [ipytone.FatOscillator() for i in range(3)]        
        self.envs = [ipytone.AmplitudeEnvelope(sync_array=True) for i in range(3)]
        self.filters = [ipytone.Filter().to_destination() for i in range(3)]
        for osc, env, filt in zip(self.oscs, self.envs, self.filters):
            osc.chain(env, filt)
            osc.start()
            
        self.drums = [ipytone.MembraneSynth(volume=-10).to_destination()
            for i in range(3)]
            
        self.seq = chaos_seq()
        
        self.octave = octave
        self.n_octaves = n_octaves
        self.octave_map = get_octave_map(octave, n_octaves)
        self.set_octave_map()

    def set_octave_map(self):
        self.octave_map = get_octave_map(self.octave, self.n_octaves)
        
    def step(self):
        projections, delta_proj, point, dist, triggers = self.seq.__next__()
        for osc, freq, det, filt in zip(self.oscs, self.octave_map(projections),
            self.detune * (1 + delta_proj), self.filters):
            osc.frequency.value = freq
            osc.detune.value = det
            filt.frequency.value = self.cut_off + self.depth * dist
                                     
        for env, trig, drum in zip(self.envs, triggers, self.drums):
            env.trigger_attack_release(self.interval * self.gate)
            if trig and not self.mute_drums:
                drum.trigger_attack_release(freq/2, self.interval * self.drum_gate)
            
        return (point, projections)
            
    def play_worker(self):
        while self.playing:
            self.step()
            time.sleep(self.interval)
            
    def start(self):
        self.playing = True
        thread = threading.Thread(target=self.play_worker)
        thread.start()
            
    def stop(self):
        self.playing = False
                                            
    def kill(self):
        for osc, env, drum in zip(self.oscs, self.envs, self.drums):
            osc.dispose()
            env.dispose()
            drum.dispose()


Turn a numpy array into [HTML hex colours](https://htmlcolorcodes.com/):

In [ ]:
def upper_hex(i):
    return ('00' + hex(i)[2:].upper())[-2:]

vhex = np.vectorize(upper_hex)

Wrap the Synth in rather a lot of [Jupyter Widgets](https://ipywidgets.readthedocs.io/en/stable/), throw in an [ipycanvas](https://ipycanvas.readthedocs.io/en/latest/) too...

In [ ]:
class ChaosGameSynthUI(ChaosGameSynth):
    
    def __init__(self, size=256):
        play_pause_button = widgets.Button(value=False,description='\u23EF')
        play_pause_button.style.font_size = '20pt'
        play_pause_button.on_click(self.play_pause)
        
        clear_button = widgets.Button(value=False,description='\u23CF')
        clear_button.style.font_size = '20pt'
        clear_button.on_click(self.clear)
        
        button_box = widgets.HBox([play_pause_button, clear_button])
        
        octave_box = widgets.widgets.BoundedIntText(value=3, min=0, max=6,
            step=1, description='octave',
            layout=widgets.Layout(width='70%'))
        octave_box.observe(self.set_octave, names='value')
        
        width_box = widgets.widgets.BoundedIntText(value=2, min=1, max=4,
            step=1, description='width',
            layout=widgets.Layout(width='70%'))
        width_box.observe(self.set_width, names='value')
        
        gate_slider = widgets.FloatSlider(value=0.8, min=0.1, max=1.0, step=0.1, 
            description='synth gate', readout=False, orientation='horizontal',
            layout=widgets.Layout(width='80%'))
        gate_slider.observe(self.set_gate, names='value')
        
        drum_gate_slider = widgets.FloatSlider(value=0.8, min=0.1, max=1.0, step=0.1, 
            description='drum gate', readout=False, orientation='horizontal',
            layout=widgets.Layout(width='80%'))
        drum_gate_slider.observe(self.set_drum_gate, names='value')
        
        mute_box = widgets.Checkbox(value=False, description='mute',
            indent=True, layout=widgets.Layout(width='80%'))
        mute_box.observe(self.set_mute_drums, names='value')
        
        v_controls = widgets.VBox([button_box, octave_box, width_box, gate_slider,
            drum_gate_slider, mute_box])
        
        bpm_slider = widgets.FloatSlider(value=240, min=30, max=360, step=10, 
            description='bpm', orientation='vertical')
        bpm_slider.observe(self.set_bpm, names='value')
        
        cut_off_slider = widgets.FloatSlider(value=200, min=100, max=500,
            step=5, description='cut-off', orientation='vertical')
        cut_off_slider.observe(self.set_cut_off, names='value')
        
        depth_slider = widgets.FloatSlider(value=100, min=0, max=400, step=20, 
            description='depth', orientation='vertical')
        depth_slider.observe(self.set_depth, names='value')
        
        detune_slider = widgets.FloatSlider(value=400, min=0, max=800, 
            step=20, description='detune', orientation='vertical') 
        detune_slider.observe(self.set_detune, names='value')
        
        attack_slider = widgets.FloatSlider(value=0.01, min=0.005, max=0.5,
            step=0.005, description='A', orientation='vertical')
        
        decay_slider = widgets.FloatSlider(value=0.1, min=0.01, max=1.0,
            step=0.01, description='D', orientation='vertical')
        
        sustain_slider = widgets.FloatSlider(value=0.75, min=0.1, max=1.0,
            step=0.1, description='S', orientation='vertical')

        release_slider = widgets.FloatSlider(value=0.5, min=0.05, max=2.0,
            step=0.1, description='R', orientation='vertical')        
        
        super(ChaosGameSynthUI, self).__init__(
            octave = octave_box.value,
            n_octaves = width_box.value,
            bpm = bpm_slider.value,
            gate = gate_slider.value,
            drum_gate = gate_slider.value,
            depth = depth_slider.value,                                  
            detune = detune_slider.value)
        
        for env in self.envs:
            widgets.link((attack_slider, 'value'), (env, 'attack'))
            widgets.link((decay_slider, 'value'), (env, 'decay'))
            widgets.link((sustain_slider, 'value'), (env, 'sustain'))
            widgets.link((release_slider, 'value'), (env, 'release'))
        
        for slider in (attack_slider, decay_slider, sustain_slider, release_slider):
            slider.observe(self.adsr_plot, names='value')

        self.adsr_box = widgets.Output()
             
        controls = widgets.HBox([v_controls, bpm_slider, cut_off_slider,
            depth_slider, detune_slider, attack_slider, decay_slider,
            sustain_slider, release_slider, self.adsr_box])
        
        self.width = size
        self.height = int(size * np.sqrt(3) / 2)
        self.canvas = Canvas(width=self.width, height=self.height)
        
        self.canvas.layout.width = '{}px'.format(2 * self.width)
        self.canvas.layout.height = "{}px".format(2 * self.height)
        
        self.clear(None)

        self.ui = widgets.VBox([controls, self.canvas])
        self.ui.layout = widgets.Layout(display='flex', flex_flow='column',
            align_items='center')
        
    def play_worker(self):
        while self.playing:
            point, proj = self.step()
            
            with hold_canvas():
                self.canvas.fill_style = '#' + ''.join(
                    vhex((255 * proj).astype(np.uint8)))
            
                x, y = (self.width * point).astype(np.int16) 
                self.canvas.fill_rect(x, self.height-y, 1)
            
            time.sleep(self.interval)
        
    def play_pause(self, _):
        if self.playing:
            self.stop()
        else:
            self.start()
            
    def clear(self, _):
        self.canvas.fill_style = "black"
        self.canvas.fill_rect(0, 0, self.width, self.height)
        
    def set_octave(self, change):
        self.octave = change.new
        self.set_octave_map()
        
    def set_width(self, change):
        self.n_octaves = change.new
        self.set_octave_map()

    def set_bpm(self, change):
        self.interval = 60.0 / change.new
        
    def set_gate(self, change):
        self.gate = change.new
        
    def set_drum_gate(self, change):
        self.drum_gate = change.new
        
    def set_mute_drums(self, change):
        self.mute_drums = change.new
        
    def set_cut_off(self, change):
        self.cut_off = change.new
        
    def set_detune(self, change):
        self.detune = change.new
        
    def set_depth(self, change):
        self.depth = change.new

    def adsr_plot(self, _):
        with plt.ioff():
            adsr_fig = plt.figure(figsize=(2,2))
            adsr_ax = adsr_fig.add_subplot(111)
            if not self.envs[0].array is None:
                adsr_ax.plot(np.arange(1024), self.envs[0].array)
        self.adsr_box.clear_output(True)
        with self.adsr_box:
            display(adsr_fig)
        plt.close(adsr_fig)
        
    def show(self):
        display(self.ui)
        self.adsr_plot(None)
        

### Here we are...

In [ ]:
synth = ChaosGameSynthUI()
synth.show()